In [ ]:
! pip install langchain
! pip install unstructured
! pip install pdf2image
! pip install pdfminer
! pip install pdfminer.six
! pip install markdown
! pip install openai
! pip install reportlab
! pip install sentence-transformers
! pip install -U langchain-community
! pip install langchain_openai
! pip install PyPDF2
! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.2 

In [5]:
import os
from langchain_community.vectorstores import FAISS
import PyPDF2
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
import configparser
import openai
import pandas as pd
from langchain_core.prompts import PromptTemplate
import json
from openai import OpenAI

In [6]:



# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the config.ini file
config.read('config.ini')

my_openai_key = config['openai']['OPENAI_API_KEY']
openai.api_key = my_openai_key
os.environ['OPENAI_API_KEY'] = my_openai_key

In [ ]:
# Load the CSV file
file_path = '/content/Student_Data_Diverse.csv'
df = pd.read_csv(file_path)

# Initialize the OpenAI API client
client = OpenAI()

# Function to generate prompt
def generate_prompt(row):
    prompt_template = PromptTemplate.from_template(
        "Create a comprehensive and personalized study plan for {name}, a {year_of_study} year student majoring in {field_of_study}. "
        "The subjects they are currently enrolled in are: {subjects}. They have a preference for {learning_styles} learning styles. "
        "Their personal objectives include: {objectives}. They are facing challenges such as: {challenges}. "
        "Additionally, they are actively involved in these extracurricular activities: {activities}. "
        "Please ensure that the study plan not only addresses their academic requirements but also aligns with their unique needs and aspirations. "
        "The plan should include specific strategies to help them achieve their objectives, overcome their challenges, and maintain a balance with their extracurricular activities."
    )
    formatted_prompt = prompt_template.format(
        name=row["Name"],
        year_of_study=row["Year of Study"],
        field_of_study=row["Field of Study"],
        subjects=row["List of Subjects"],
        learning_styles=row["Preferred Learning Styles"],
        objectives=row["Personal Objectives"],
        challenges=row["Challenges"],
        activities=row["Extracurricular Activities"]
    )
    return formatted_prompt

# Loop to repeatedly ask for student names and generate study plans
responses = {}
while True:
    # Prompt the user to input a name
    student_name = input("Enter the name of the student (or type 'exit' to quit): ")

    if student_name.lower() == 'exit':
        break

    # Check if the entered name exists in the DataFrame
    if student_name in df['Name'].values:
        # Get the row corresponding to the entered name
        row = df[df['Name'] == student_name].iloc[0]

        # Generate the prompt for the selected student
        prompt = generate_prompt(row)

        # Highlighting the structure and its importance
        print("Generated Prompt Structure:\n")
        print(f"1. Introduction and Context: Clearly states the need for a comprehensive and personalized study plan for {row['Name']}.")
        print("2. Academic Details: Lists the subjects the student is enrolled in to ensure the plan addresses their academic requirements.")
        print("3. Learning Preferences: Mentions the student's preferred learning styles to tailor the study methods accordingly.")
        print("4. Personal Objectives: Includes the student's goals to ensure the plan aligns with their aspirations.")
        print("5. Challenges: Identifies the student's challenges to provide targeted strategies for overcoming them.")
        print("6. Extracurricular Activities: Considers the student's extracurricular involvement to balance academic and personal life.")
        print("7. Comprehensive and All-Inclusive Plan: Ensures the study plan is holistic, addressing all aspects of the student's academic and personal development.")
        print("\n")

        # Create a study plan using OpenAI API
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI assistant named Bob. Your role is to create personalized study plans for students. Carefully consider the provided details, including their academic subjects, learning preferences, personal goals, challenges, and extracurricular activities, to generate a comprehensive and effective study plan tailored to their needs."},
                {"role": "user", "content": prompt},
            ],
        )

        # Get the response content
        response_content = response.choices[0].message.dict()["content"]
        print(response_content)

        # Store the response in the dictionary
        responses[student_name] = response_content
    else:
        print("The entered name does not exist in the CSV file.")

# Save all responses to a JSON file
with open('student_study_plans.json', 'w') as json_file:
    json.dump(responses, json_file, indent=4)

print("All responses have been saved to 'student_study_plans.json'.")

Enter the name of the student (or type 'exit' to quit): Tahawar
Generated Prompt Structure:

1. Introduction and Context: Clearly states the need for a comprehensive and personalized study plan for Tahawar.
2. Academic Details: Lists the subjects the student is enrolled in to ensure the plan addresses their academic requirements.
3. Learning Preferences: Mentions the student's preferred learning styles to tailor the study methods accordingly.
4. Personal Objectives: Includes the student's goals to ensure the plan aligns with their aspirations.
5. Challenges: Identifies the student's challenges to provide targeted strategies for overcoming them.
6. Extracurricular Activities: Considers the student's extracurricular involvement to balance academic and personal life.
7. Comprehensive and All-Inclusive Plan: Ensures the study plan is holistic, addressing all aspects of the student's academic and personal development.


### Comprehensive and Personalized Study Plan for Tahawar

#### Overvie